In [1]:
# Pacotes de trabalho
import pandas as pd
import numpy as np
from datetime import datetime
import itertools
from itertools import combinations
import efficient_apriori
from efficient_apriori import apriori
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Carga de dados
df = pd.read_csv("dados.csv", sep=';', encoding='utf-8')
# Criando um dataframe para armazenar os produtos com id e nome
df_produto = df[['product_id', 'product_name']].drop_duplicates()

In [3]:
def modelo_associacao(min_support,min_confidence):
    # Inicia a preparação dos dados agrupando os dados pela ordem de venda e os IDs dos produtos
    df_preparacao = pd.DataFrame(df.groupby('order_id')['product_id'])
    # Cria uma estrutura de lista
    prod = []
    for i in range(len(df_preparacao[0])):
        prod.append(list(df_preparacao.iloc[i][1]))
    prod_ = prod
    order_ = df_preparacao[0]
    
    # Prepara o dataframe
    transactions = pd.DataFrame({'OrderID' : order_, 'Products' : prod_})
    # Prepara a tupla com as transações
    transactions_tup = [tuple(row) for row in transactions['Products'].tolist()]
    
    # Cria ae regra de associação
    itemsets_ap, rules_ap = apriori(transactions_tup, min_support, min_confidence)
    
    return rules_ap

In [4]:
# Parametrização
min_support = 0.0005
min_confidence = 0.1

In [5]:
# Chamada a função para criar o modelo de regra de assosciação
rules_ap = modelo_associacao(min_support,min_confidence)

In [6]:
# Exibe o resultado de cada associação e será necessário organizar em formatado de tabela.
for x in rules_ap:
    print(x)

{4} -> {2} (conf: 0.270, supp: 0.002, lift: 7.376, conv: 1.319)
{5} -> {2} (conf: 0.252, supp: 0.002, lift: 6.879, conv: 1.287)
{6} -> {2} (conf: 0.189, supp: 0.004, lift: 5.178, conv: 1.188)
{2} -> {6} (conf: 0.107, supp: 0.004, lift: 5.178, conv: 1.096)
{8} -> {2} (conf: 0.202, supp: 0.005, lift: 5.530, conv: 1.208)
{2} -> {8} (conf: 0.150, supp: 0.005, lift: 5.530, conv: 1.145)
{9} -> {2} (conf: 0.185, supp: 0.001, lift: 5.060, conv: 1.182)
{10} -> {2} (conf: 0.133, supp: 0.001, lift: 3.646, conv: 1.112)
{11} -> {2} (conf: 0.224, supp: 0.001, lift: 6.126, conv: 1.242)
{12} -> {2} (conf: 0.175, supp: 0.003, lift: 4.793, conv: 1.168)
{15} -> {2} (conf: 0.159, supp: 0.003, lift: 4.360, conv: 1.146)
{16} -> {2} (conf: 0.152, supp: 0.003, lift: 4.149, conv: 1.136)
{18} -> {2} (conf: 0.143, supp: 0.003, lift: 3.912, conv: 1.124)
{19} -> {2} (conf: 0.181, supp: 0.003, lift: 4.946, conv: 1.176)
{21} -> {2} (conf: 0.165, supp: 0.004, lift: 4.517, conv: 1.154)
{2} -> {21} (conf: 0.111, supp: 

In [7]:
def formata_resultado(rules_ap):
    # Extrai o produto A, produto B, Confidence, Support e Lift
    df_ajustado = pd.DataFrame(rules_ap)
    df_ajustado.rename(columns={0:"Dados"}, inplace=True)
    df_ajustado['Dados'] = df_ajustado['Dados'].astype(str)
    df_ajustado['novo'] = df_ajustado['Dados'].str.replace(' -> ','').str.replace('}',',').str.replace('{','').str.replace('(','').str.replace(')','')
    divisao = df_ajustado['novo'].str.split(',')
    ProductA = divisao.str.get(0)
    ProductB = divisao.str.get(1)
    Confidence = divisao.str.get(2)
    Suport = divisao.str.get(3)
    Lift = divisao.str.get(4)
    
    # Cria um data frame extraindo as métricas
    df_final = pd.DataFrame()
    df_final['ProductA'] = ProductA
    df_final['ProductB'] = ProductB
    df_final['Confidence'] = Confidence
    df_final['Suport'] = Suport
    df_final['Lift'] = Lift
    df_final['Confidence'] = df_final['Confidence'].astype(str)
    df_final['Confidence'] = df_final['Confidence'].str.replace('conf: ','')
    df_final['Suport'] = df_final['Suport'].astype(str)
    df_final['Suport'] = df_final['Suport'].str.replace('supp: ','')
    df_final['Lift'] = df_final['Lift'].astype(str)
    df_final['Lift'] = df_final['Lift'].str.replace('lift: ','')
    
    # Removendo combinações repetidas
    df_final['conf'] = df_final['ProductA'] + df_final['ProductB']
    df_final = df_final.drop_duplicates(subset='conf')
    df_final['conf'] = df_final['ProductA'] == df_final['ProductB']
    df_final = df_final.query('conf == False')
       
    # Formatando o Dataset final
    df_produto_a = df_produto.rename(columns={'product_id':'ProductA'})
    df_produto_a['ProductA'] = df_produto_a['ProductA'].astype('str')
    df_final = df_final.merge(df_produto_a, on = 'ProductA', how = 'inner')
    df_final['Product_A'] = df_final['product_name']
    df_final = df_final.drop(columns = ['ProductA','product_name'], axis = 1)
    df_produto_b = df_produto.rename(columns={'product_id':'ProductB'})
    df_produto_b['ProductB'] = df_produto_b['ProductB'].astype('str')
    df_final = df_final.merge(df_produto_b, on = 'ProductB', how = 'inner')
    df_final['Product_B'] = df_final['product_name']
    df_final = df_final.drop(columns = ['ProductB','product_name'], axis = 1)
    df_final = df_final[['Product_A','Product_B','Suport','Confidence','Lift']]
    df_final['Suport'] = df_final['Suport'].str.replace(' ','')
    df_final['Suport'] = df_final['Suport'].astype('float')
    df_final['Confidence'] = df_final['Confidence'].str.replace(' ','')
    df_final['Confidence'] = df_final['Confidence'].astype('float')
    df_final['Lift'] = df_final['Lift'].str.replace(' ','')
    df_final['Lift'] = df_final['Lift'].astype('float')
    df_final = df_final[['Product_A','Product_B','Confidence','Lift']].sort_values(by = 'Lift', ascending = False)
    
    return df_final

In [8]:
# Chamada a função para formatar a saída dos modelos e conseguirmos analisar os dados em formato de tabela
df_modelo_final = formata_resultado(rules_ap)

In [9]:
# Visualizando o relatório final das 50 regras mais forte com base no lift
df_modelo_final.head(50)

,Product_A,Product_B,Confidence,Lift
1882,LINGUICA CALABRESA CIACARNE KG,BACON DEF EXTRA PERNIL CIACARNE KG,0.250,81.246
1837,BACON DEF EXTRA PERNIL CIACARNE KG,LINGUICA CALABRESA CIACARNE KG,0.188,81.246
1858,HOT DOG ALEMAO,BRIOCHE DE FRANGO,0.139,28.174
1871,BRIOCHE DE FRANGO,HOT DOG ALEMAO,0.128,28.174
1123,SALSA,CEBOLINHA,0.528,25.608
1382,CEBOLINHA,SALSA,0.381,25.608
1856,BRIOCHE DE CALABRESA,BRIOCHE DE FRANGO,0.121,24.673
1868,BRIOCHE DE FRANGO,BRIOCHE DE CALABRESA,0.162,24.673
1814,PIMENTAO MADURO,BERINJELA,0.144,23.682
1815,PIMENTAO MADURO,PIMENTAO VERDE,0.196,22.585


In [10]:
df_modelo_final.QUERY['Product_A == "Product_A"'].head(10)

AttributeError: 'DataFrame' object has no attribute 'QUERY'